# Hybrid Recommender System
### combine content-based and collaborative filtering

In [1]:
import pandas as pd
import numpy as np
import pickle
from surprise import KNNWithMeans

In [2]:
# load cosine_matrix.pkl
with open('cosine_matrix.pkl', 'rb') as f:
    cosine_matrix = pickle.load(f)

In [3]:
# load knn.pkl
with open('knn_matrix.pkl', 'rb') as f:
    knn_matrix = pickle.load(f)
    

In [4]:
# load titles.pkl
with open('titles.pkl', 'rb') as f:
    titles = pickle.load(f)

#### weighted matrix

In [5]:
# change weights if desired
knn_sim_weight = 0.5
cos_sim_weight = 0.5

In [6]:
weighted_sim = (knn_sim_weight * knn_matrix) + (cos_sim_weight * cosine_matrix)

### Retrieving top recommendations

In [7]:
titles.head()

,track_id,name,artist
0,0,Mr. Brightside,The Killers
1,1,Wonderwall,Oasis
2,2,Take Me Out,Franz Ferdinand
3,3,Karma Police,Radiohead
4,4,Clocks,Coldplay


In [13]:
def get_track_id(artist, song):
    '''Get index from titles where artist and song match'''
    
    try:
        track_id = titles[(titles['artist'] == artist) & (titles['name'] == song)].index[0]
        return track_id
    except:
        print('Song not found in database')
        return None

In [14]:
# test
get_track_id('Oasis', 'Wonderwall')

1

In [16]:
def get_song(index):
    '''Get song artist and name at index'''
    try:
        row = titles.iloc[index]
        artist_song = row['artist'] + ' - ' + row['name']
        return artist_song
    except:
        print('Index not found in database')
        return None

In [17]:
# test
get_song(0)

'The Killers - Mr. Brightside'

In [18]:
def recommend_content(artist, title, sim_matrix):
    '''Get similar songs using the similarity matrix'''
    # get index for our song
    idx = get_track_id(artist, title)
    
    # get similarity scores of all songs w.r.t to our song
    sim_scores = list(enumerate(sim_matrix[idx]))
    
    # sort scores based on similarity
    sorted_sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # limit to 20 songs
    sorted_sim_scores = sorted_sim_scores[1:20]

    # get song artist and names
    content_similar_scores = []
    for i in sorted_sim_scores:
        content_similar_scores.append(get_song(i[0]))
    
    return content_similar_scores

In [19]:
# recommend_content test
recommend_content('Oasis', 'Wonderwall', weighted_sim)

['Placebo - This Picture',
 'Tonic - You Wanted More',
 "Snow Patrol - You're All I Have",
 'Pulp - Common People',
 'The All-American Rejects - My Paper Heart',
 'The Fratellis - Vince The Loveable Stoner',
 'The Vaccines - If You Wanna',
 "Oasis - Don't Look Back in Anger",
 "The Libertines - Can't Stand Me Now",
 'Jarvis Cocker - Fat Children',
 'Kaiser Chiefs - Na Na Na Na Naa',
 'Arctic Monkeys - This House Is a Circus',
 'Biffy Clyro - Mountains',
 'The Kooks - Gap',
 "Robbie Williams - Won't Do That",
 'Boys Like Girls - Five Minutes To Midnight',
 'Robbie Williams - Random Acts Of Kindness',
 'The Kooks - See the Sun',
 'Scouting for Girls - The Mountains Of Navaho']

In [21]:
# store weighted_sim in pickle
with open('weighted_sim.pkl', 'wb') as f:
    pickle.dump(weighted_sim, f)